# Etapa 1: Importação das bibliotecas

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=52a3fed770b4df7be310ce24100ffa4cd9ce6fb21f9e56efd3562303ec0ad6bf
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=28ce1ada2821acaeabf2ed85a4219034c411286e61b8948ba637736bff045af7
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=f085541050d96656a1f5e85e8e7a5bc09ef180680c3e8af572bb8e0279f33360
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 10.8MB/s 


In [ ]:
!pip install tensorflow==2.2.0-rc3

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 3.0MB 37.3MB/s 
     |████████████████████████████████| 460kB 34.1MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
tf.__version__
# !pip install tensorflow==2.2.0-rc3

'2.2.0-rc3'

# Etapa 2: Pré-processamento

## Carregamento dos arquivos

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/IA MachineLearning DataSciense/BERT-classificacao_sentimentos/training.1600000.processed.noemoticon.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
data.shape

(1600000, 2)

In [ ]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Pré-processamento

### Limpeza

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenização

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False) #BERT smool. não é o large
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case) #vocab_file = todas as palavras em ingles, do_lower_case = 

In [ ]:
vocab_file

b'/tmp/tfhub_modules/03d6fb3ce1605ad9e5e9ed5346b2fb9623ef4d3d/assets/vocab.txt'

In [ ]:
do_lower_case

True

In [ ]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]
#CLS = token para classificação
#SEP = token de separação para a sentença B. Mesmo que não tenha uma sentença B, é preciso colocar

In [ ]:
encode_sentence("My dog likes strawberries.")
#essa será a formatação para a base de dados, ou seja, o input para o algoritmo BERT

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [ ]:
#para cada registro do texto da base de dados, faz a separação, como antes
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [ ]:
#buscando os dois primeiros registros
print(data_inputs[0:2])

[['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


### Criação da base de dados

MODELO RECEBE 3 PARÂMETROS DIFERENTES, não apenas só as frases:

PRECISA RECEBER:

1)converte o id dos tokens: valores dos tokens

2)máscara: onde está o token de padding: é preciso que todas as frases tenham o mesmo tamanho. Logo, se uma frase tiver 5 tokens e outra 8 tokens, é preciso preencher a sentença de 5 tokens para 8 tokens, adicionando caracter especial chamado padding, retornando 0:false ou 1:true

3)Lista de 0: tokens da primeira sentença; 1: valores da segunda sentença. Maneira de indicar quando uma sentença começa ou termica
My dog is  SEP My cat is
0   0   0      1   1   1



In [ ]:
#1º TIPO DE ENTRADA
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
get_ids(tokenizer.tokenize("My dog likes strawberries."))

[2026, 3899, 7777, 13137, 20968, 1012]

In [ ]:
#compara cada token da sentença com o caracter PAD
np.char.not_equal("[PAD]", "[PAD]") #não são diferentes

array(False)

In [ ]:
#2º TIPO DE ENTRADA
def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)   #converte para o tipo inteiro o true ou false
#[PAD] = indica que haverá um tipo de preenchimento se as sentenças não forem iguais em tamanho

In [ ]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))
#nessa sentença, não tem o token PAD. Se tivesse o token pad, então seria colocado 0

array([1, 1, 1, 1, 1, 1])

In [ ]:
#3º TIPO DE ENTRADA
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0              #primeira sentença com valor 0
  for tok in tokens:              #percorre cada valor dos tokens
    seg_ids.append(current_seg_id) 
    if tok == "[SEP]":            #quando tiver um separador, SEP, entre as sentenças, então coloca na lista valor 1
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [ ]:
#primeira sentenças com os tokens
print(data_inputs[0])

['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [ ]:
#sequencia de 0: indica que é a primeira sentença
get_segments(data_inputs[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']

In [ ]:
#adiciona 3 camadas de INPUT
bert_layer([
            tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),     #expande as dimensões precisando estar no formato de BATCH
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0) 
           ])
#tf.cast() = converte de tipo
#tf.cast(get_ids(my_sent), tf.int32), 0) = converte os ids da sentença para int32 e0 pois coloca uma dimensão a mais no início

#RETORNA 2 MATRIZ DE EMBEDDING: esse retorno é o final, ou seja, escolher qual retorno quer/precisa
'''
  RETORNO 1: shape=(1, 768) = usado para a CLASSIFICAÇÃO   -> 1 registro para 768 colunas
  RETORNO 2: shape=(1, 6, 768) = 1 registro/frase com 6 palavras e 768 colunas
                                nível de token, representação de cada palavra
 '''
#shape=(1, 768) = 1: tem uma sentença, resultado do valor 0 colocado no final, adicionaod uma dimensão
#                 768: matriz de embedding das colunas          

#shape=(1, 6, 768) = 6: tem seis tokens nessa frase

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935541e-01, -4.10335153e-01, -9.65755045e-01,
          9.07317877e-01,  8.12913775e-01, -1.74174339e-01,
          9.11234558e-01,  3.41952085e-01, -8.74521315e-01,
         -9.99989271e-01, -7.78409958e-01,  9.69385207e-01,
          9.86160457e-01,  6.36963248e-01,  9.48631227e-01,
         -7.51193166e-01, -4.58339661e-01, -7.08104372e-01,
          4.62098330e-01, -6.57927096e-01,  7.60414422e-01,
          9.99994814e-01, -3.96861106e-01,  3.44166011e-01,
          6.16488576e-01,  9.94400024e-01, -7.76633739e-01,
          9.38316464e-01,  9.59452212e-01,  7.32879102e-01,
         -6.93436623e-01,  2.93080389e-01, -9.93785501e-01,
         -1.64551780e-01, -9.67019558e-01, -9.95549500e-01,
          5.32935381e-01, -6.88060820e-01,  1.34715075e-02,
          2.98195817e-02, -9.18356419e-01,  4.20526296e-01,
          9.99988914e-01,  2.52676070e-01,  6.06235385e-01,
         -3.50750059e-01, -1.00000000e+00,  4.975

[-0.07947513,  0.00580787, -0.31414014, ..., -0.45097244,
           0.29333133,  0.23387688],

         [ 0.3931604 ,  0.5033638 ,  0.24021432, ..., -0.32635677,
           0.3498609 ,  0.20673229],

         [ 0.35789207,  0.10767116, -0.04988845, ..., -0.5082272 ,
           0.25048867, -0.26268798],

         [-0.2989215 , -0.24708748,  0.07151444, ..., -0.33810034,
           0.12699437, -0.09681879],

         [-0.36815315, -0.71465284, -0.2103259 , ...,  0.35395196,
           0.3343857 , -0.6233478 ],

         [ 0.88692284, -0.16996934, -0.29173723, ...,  0.0581655 ,
          -0.57759917, -0.3207531 ]

shape=(1,6,768):

Para cada palavra dessa frase, ou seja, as 6 palavras de 1 frase, cada palavra tem um embedding de 768 colunas

ESSA É A REPRESENTAÇÃO QUE SERÁ CONSTRUÍDO O MODELO

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0)) # 0 = token de padding

In [ ]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Etapa 3: Construção do modelo

In [ ]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    #construtor
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        #camada de embedding;      treinable = false: não será treinado o modelo, apenas usado 16min:35s aula 10
        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                         trainable = False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    #camada de embedding: BERT será uma camada de embedding no treinamento
    def embed_with_bert(self, all_tokens):
      #_, emb = : _ => descarta o que o all_tokens retorna por primeiro, que é a representação da frase inteira e pega apenas o 2º vetor: shape=(1,6,768)
      _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
      return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        #concatena  as 3 camadas convolucionais de trainamento
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Etapa 4: Treinamento

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
#rede neural convolucional
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/IA MachineLearning DataSciense/BERT-classificacao_sentimentos/sentimentos"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
#salva o modeo a cada época
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   steps_per_epoch = 200,
                   callbacks=[MyCustomCallback()])

Epoch 1/5
200/200 [==============================] - 16s 80ms/step - loss: 0.3232 - accuracy: 0.8659
Epoch 2/5
200/200 [==============================] - 16s 80ms/step - loss: 0.3231 - accuracy: 0.8645
Epoch 3/5
200/200 [==============================] - 16s 81ms/step - loss: 0.3163 - accuracy: 0.8648
Epoch 4/5
200/200 [==============================] - 16s 81ms/step - loss: 0.3151 - accuracy: 0.8683
Epoch 5/5
200/200 [==============================] - 16s 80ms/step - loss: 0.3056 - accuracy: 0.8737


Epoch 1/5
  40623/Unknown - 2999s 74ms/step - loss: 0.3973 - accuracy: 0.8219Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 3001s 74ms/step - loss: 0.3973 - accuracy: 0.8219
Epoch 2/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3756 - accuracy: 0.8339Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2987s 74ms/step - loss: 0.3756 - accuracy: 0.8339
Epoch 3/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3659 - accuracy: 0.8391Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2979s 73ms/step - loss: 0.3659 - accuracy: 0.8391
Epoch 4/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3586 - accuracy: 0.8426Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2969s 73ms/step - loss: 0.3586 - accuracy: 0.8426
Epoch 5/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3527 - accuracy: 0.8454Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2960s 73ms/step - loss: 0.3527 - accuracy: 0.8454
<tensorflow.python.keras.callbacks.History at 0x7fd363ef4080>

# Etapa 5: Avaliação do modelo

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 268s 59ms/step - loss: 0.3259 - accuracy: 0.8619
[0.325869619846344, 0.8618712425231934]


In [ ]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  #empilha as camadas
  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0) #0 indicação de 1 registro

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))


In [ ]:
get_prediction("This actor is very bad.")

Output of the model: [[0.0178184]]
Predicted sentiment: negative


In [ ]:
get_prediction("I like dogs.")

Output of the model: [[0.9603923]]
Predicted sentiment: positive
